# Tekstiluokittelutehtävä

Tässä moduulissa aloitamme yksinkertaisella tekstiluokittelutehtävällä, joka perustuu **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**-aineistoon: luokittelemme uutisotsikot yhteen neljästä kategoriasta: Maailma, Urheilu, Liiketoiminta ja Tiede/Tekniikka.

## Aineisto

Aineiston lataamiseen käytämme **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**-rajapintaa.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Voimme nyt käyttää aineiston harjoitus- ja testiosia käyttämällä `dataset['train']` ja `dataset['test']` vastaavasti:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Tulostetaan ensimmäiset 10 uutta otsikkoa aineistostamme:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Tekstin vektorisointi

Nyt meidän täytyy muuntaa teksti **numeroiksi**, jotka voidaan esittää tensoreina. Jos haluamme sanatasoisen esityksen, meidän täytyy tehdä kaksi asiaa:

* Käyttää **tokenisoijaa** jakamaan teksti **tokeneiksi**.
* Rakentaa näistä tokeneista **sanasto**.

### Sanaston koon rajoittaminen

AG News -aineiston esimerkissä sanaston koko on melko suuri, yli 100 000 sanaa. Yleisesti ottaen emme tarvitse sanoja, jotka esiintyvät tekstissä harvoin — vain muutamassa lauseessa niitä on, eikä malli opi niistä. Siksi on järkevää rajoittaa sanaston koko pienemmäksi antamalla argumentti vektorisointikerroksen konstruktorille:

Molemmat näistä vaiheista voidaan hoitaa käyttämällä **TextVectorization**-kerrosta. Luodaan vektorisointiobjekti ja kutsutaan sitten `adapt`-metodia, jotta käydään läpi kaikki teksti ja rakennetaan sanasto:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Huomaa**, että käytämme vain osajoukkoa koko aineistosta sanaston rakentamiseen. Teemme tämän nopeuttaaksemme suoritusaikaa, jotta sinun ei tarvitse odottaa. Otamme kuitenkin riskin, että jotkut sanat koko aineistosta eivät sisälly sanastoon ja ne ohitetaan koulutuksen aikana. Koko sanaston koon käyttäminen ja koko aineiston läpikäyminen `adapt`-vaiheen aikana voisi parantaa lopullista tarkkuutta, mutta ei merkittävästi.

Nyt voimme käyttää varsinaista sanastoa:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Vektorisoijaa käyttämällä voimme helposti koodata minkä tahansa tekstin numerosarjaksi:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Bag-of-words-tekstin esitys

Koska sanat edustavat merkitystä, joskus voimme ymmärtää tekstin merkityksen pelkästään tarkastelemalla yksittäisiä sanoja, riippumatta niiden järjestyksestä lauseessa. Esimerkiksi uutisia luokitellessa sanat kuten *sää* ja *lumi* viittaavat todennäköisesti *sääennusteeseen*, kun taas sanat kuten *osakkeet* ja *dollari* liittyvät *talousuutisiin*.

**Bag-of-words** (BoW) -vektoriesitys on perinteisistä vektoriesityksistä yksinkertaisin ymmärtää. Jokainen sana yhdistetään vektorin indeksiin, ja vektorin elementti sisältää kunkin sanan esiintymiskertojen määrän tietyssä dokumentissa.

![Kuva, joka näyttää, miten bag-of-words-vektoriesitys tallennetaan muistiin.](../../../../../translated_images/fi/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Voit myös ajatella BoW:n olevan summa kaikista yksittäisten sanojen yksi-kuuma-koodatuista vektoreista tekstissä.

Alla on esimerkki siitä, miten bag-of-words-esitys voidaan luoda Scikit Learn -python-kirjaston avulla:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Voimme myös käyttää yllä määrittelemäämme Keras-vektoroijaa, muuntaen jokaisen sanan numeron yksi-hot-koodaukseksi ja yhteenlaskemalla kaikki nämä vektorit:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Huom**: Saatat yllättyä, että tulos poikkeaa aiemmasta esimerkistä. Syynä on se, että Keras-esimerkissä vektorin pituus vastaa sanaston kokoa, joka rakennettiin koko AG News -aineistosta, kun taas Scikit Learn -esimerkissä rakensimme sanaston lennossa näytetekstistä.


## BoW-luokittelijan kouluttaminen

Nyt kun olemme oppineet rakentamaan tekstimme bag-of-words-esityksen, koulutetaan luokittelija, joka käyttää sitä. Ensin meidän täytyy muuntaa datamme bag-of-words-esitykseksi. Tämä voidaan tehdä käyttämällä `map`-funktiota seuraavalla tavalla:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Nyt määritellään yksinkertainen luokittelijaneuroverkko, joka sisältää yhden lineaarisen kerroksen. Syötteen koko on `vocab_size`, ja ulostulon koko vastaa luokkien määrää (4). Koska ratkaistaan luokittelutehtävää, lopullinen aktivointifunktio on **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Koska meillä on 4 luokkaa, yli 80 %:n tarkkuus on hyvä tulos.

## Luokittelijan kouluttaminen yhtenä verkostona

Koska vektoroija on myös Keras-kerros, voimme määritellä verkoston, joka sisältää sen, ja kouluttaa sen päästä päähän. Tällä tavalla meidän ei tarvitse vektoroida datasettiä käyttämällä `map`-funktiota, vaan voimme yksinkertaisesti syöttää alkuperäisen datasetin verkoston syötteeksi.

> **Note**: Meidän täytyisi silti soveltaa map-toimintoja datasettiimme, jotta voimme muuntaa sanakirjojen kentät (kuten `title`, `description` ja `label`) tupleiksi. Kuitenkin, kun lataamme dataa levyltä, voimme alusta alkaen rakentaa datasetin tarvittavalla rakenteella.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrammit, trigrammit ja n-grammit

Yksi bag-of-words-lähestymistavan rajoitus on, että jotkin sanat kuuluvat monisanaisiin ilmauksiin. Esimerkiksi sana 'hot dog' tarkoittaa jotain täysin erilaista kuin sanat 'hot' ja 'dog' muissa yhteyksissä. Jos edustamme sanoja 'hot' ja 'dog' aina samoilla vektoreilla, se voi hämmentää malliamme.

Tämän ratkaisemiseksi käytetään usein **n-grammi-edustuksia** dokumenttien luokittelumenetelmissä, joissa jokaisen sanan, kahden sanan yhdistelmän tai kolmen sanan yhdistelmän esiintymistiheys on hyödyllinen ominaisuus luokittelijoiden kouluttamisessa. Esimerkiksi bigrammi-edustuksissa lisäämme sanastoon kaikki sanaparit alkuperäisten sanojen lisäksi.

Alla on esimerkki siitä, miten bigrammi bag-of-words -edustus voidaan luoda käyttämällä Scikit Learnia:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram-lähestymistavan suurin haitta on, että sanaston koko alkaa kasvaa erittäin nopeasti. Käytännössä meidän täytyy yhdistää n-gram-esitys ulottuvuuksien vähentämistekniikkaan, kuten *upotuksiin*, joita käsittelemme seuraavassa osiossa.

Jotta voimme käyttää n-gram-esitystä **AG News** -aineistossamme, meidän täytyy välittää `ngrams`-parametri `TextVectorization`-rakentajalle. Bigram-sanaston pituus on **merkittävästi suurempi**, meidän tapauksessamme yli 1,3 miljoonaa tokenia! Siksi on järkevää rajoittaa myös bigram-tokenit kohtuulliseen määrään.

Voisimme käyttää samaa koodia kuin yllä luokittelijan kouluttamiseen, mutta se olisi erittäin muistitehotonta. Seuraavassa osiossa koulutamme bigram-luokittelijan käyttämällä upotuksia. Sillä välin voit kokeilla bigram-luokittelijan kouluttamista tässä muistikirjassa ja katsoa, saatko paremman tarkkuuden.


## BoW-vektoreiden automaattinen laskeminen

Yllä olevassa esimerkissä laskimme BoW-vektorit käsin yhteenlaskemalla yksittäisten sanojen yksi-kuuma-koodaukset. Uusin TensorFlow-versio mahdollistaa kuitenkin BoW-vektoreiden automaattisen laskemisen välittämällä `output_mode='count`-parametrin vektorisoinnin konstruktoriin. Tämä tekee mallin määrittelystä ja kouluttamisesta huomattavasti helpompaa:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Term frequency - inverse document frequency (TF-IDF)

BoW-esityksessä sanan esiintymisiä painotetaan samalla tavalla riippumatta itse sanasta. On kuitenkin selvää, että yleiset sanat kuten *a* ja *in* ovat paljon vähemmän merkityksellisiä luokittelun kannalta kuin erikoistuneet termit. Useimmissa NLP-tehtävissä jotkut sanat ovat tärkeämpiä kuin toiset.

**TF-IDF** tarkoittaa **term frequency - inverse document frequency**. Se on muunnelma bag-of-words-menetelmästä, jossa binäärisen 0/1-arvon sijaan, joka ilmaisee sanan esiintymisen dokumentissa, käytetään liukulukuarvoa, joka liittyy sanan esiintymisen tiheyteen korpuksessa.

Tarkemmin määriteltynä sanan $i$ paino $w_{ij}$ dokumentissa $j$ määritellään seuraavasti:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
missä
* $tf_{ij}$ on sanan $i$ esiintymiskertojen määrä dokumentissa $j$, eli BoW-arvo, jonka olemme aiemmin nähneet
* $N$ on kokoelman dokumenttien lukumäärä
* $df_i$ on niiden dokumenttien lukumäärä, jotka sisältävät sanan $i$ koko kokoelmassa

TF-IDF-arvo $w_{ij}$ kasvaa suhteessa siihen, kuinka monta kertaa sana esiintyy dokumentissa, ja sitä tasapainotetaan korpuksen dokumenttien määrällä, jotka sisältävät kyseisen sanan. Tämä auttaa korjaamaan sen, että jotkut sanat esiintyvät useammin kuin toiset. Esimerkiksi, jos sana esiintyy *jokaisessa* kokoelman dokumentissa, $df_i=N$, ja $w_{ij}=0$, jolloin nämä termit jätetään kokonaan huomiotta.

Voit helposti luoda tekstin TF-IDF-vektorisoinnin käyttämällä Scikit Learnia:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Kerasissa `TextVectorization`-kerros voi automaattisesti laskea TF-IDF-taajuudet käyttämällä `output_mode='tf-idf'`-parametria. Toistetaan yllä käyttämämme koodi nähdäksemme, lisääkö TF-IDF:n käyttö tarkkuutta:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Johtopäätös

Vaikka TF-IDF-esitykset antavat sanakohtaisia painotuksia niiden esiintymistiheyden perusteella, ne eivät pysty ilmaisemaan merkitystä tai järjestystä. Kuten kuuluisa kielitieteilijä J. R. Firth totesi vuonna 1935: "Sanalla on aina täydellinen merkitys vain kontekstissaan, eikä merkityksen tutkimista ilman kontekstia voida ottaa vakavasti." Kurssin myöhemmässä vaiheessa opimme, kuinka tekstistä voidaan saada kontekstuaalista tietoa kielenmallinnuksen avulla.



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
